In [35]:
using Pkg
using Distributions

using Optim
using PyPlot
using LinearAlgebra

In [49]:
function market_shares1(p,alpha)
    return exp.(-alpha.*p) / (1 + sum( exp.(-alpha.*p) ) )
end


function mco_profits1(p,phi,mu,alpha)
    return market_shares1(p,alpha) .* ( p - mu .* phi )
end


function price_profits1(phi,mu,alpha)
    diff =  5.
    p = [1.]
    profits = [0.]
    
    #set up objective
    obj1(p0) = -1*mco_profits1( [p0[1] ], phi,mu,alpha)[1]
    res1 = optimize(obj1, [p[1]] , BFGS())

    #update result
    p = res1.minimizer[1]
    profits =  [-1*res1.minimum]        
    return p, profits
end


#arbitrary outside option... passive beliefs
function nash_in_nash1(phi, mu, alpha; mc=0., pi0=0.)
    p, profits = price_profits1(phi,mu,alpha)
    s = market_shares1(p,alpha)
    return  profits .* (sum(mu .* s .* (phi .- mc) ) - pi0)
end


function simult_bargain1(mu, alpha; mc=0.)
    """solve each firm 1s optimization holding theta 2 fixed"""
    phi = alpha
    obj1(phi0) = -1*nash_in_nash1( [phi0[1] ] , mu, alpha ; mc=mc)[1]
    res1 =  optimize(obj1, [phi[1]], BFGS())

    return [ res1.minimizer[1] ]
end

practice_alpha =.6
println(price_profits1(nash_in_nash1([3.], [.5] , practice_alpha),[.5],practice_alpha))

(2.145662699808205, [0.45998])


In [61]:
function market_shares(p,alpha)
    return exp.(-alpha.*p) / (1 + sum( exp.(-alpha.*p) ) )
end


function mco_profits(p,phi,mu,alpha)
    return market_shares(p,alpha) .* ( p - mu .* phi )
end


function price_profits(phi,mu,alpha)
    diff =  5.
    p = [1.,1.]
    profits = [0.,0.]
    
    maxiter = 10
    while (maxiter >=0) & (diff > 10e-7)
        #set up objective
        obj1(p0) = -1*mco_profits( [p0[1], p[2]], phi,mu,alpha)[1]
        res1 = optimize(obj1, [p[1]] , BFGS())
        
        obj2(p0) = -1*mco_profits( [p[1], p0[1]] ,phi,mu,alpha)[2]
        res2 = optimize(obj2,[p[2]], BFGS())
        
        #update loop variables
        diff = max( abs( res1.minimizer[1] - p[1] ), abs( res2.minimizer[1] - p[2] ) )
        maxiter = maxiter-1
        
        #update result
        p = [res1.minimizer[1],res2.minimizer[1]]
        profits =  [-1*res1.minimum, -1*res2.minimum]        
    end
    return p, profits
end


#arbitrary outside option... passive beliefs
function nash_in_nash(phi, mu, alpha; mc=0., pi0=0.)
    p, profits = price_profits(phi,mu,alpha)
    s = market_shares(p,alpha)
    return  profits .* (sum(mu .* s .* (phi .- mc) ) - pi0)
end

print( nash_in_nash( [1000.,1000.], [.5,.5] , .5 ;pi0=.55) )

[-2.7163e-269, -2.7163e-269]

In [66]:
function bargain_helper(phi, mu, alpha)
    """solve each firm 1s optimization holding theta 2 fixed"""
    
    phi01 = simult_bargain1(mu[1], alpha)
    pi01 = price_profits1(phi01,mu[1],alpha)[2][1]
    phi02 = simult_bargain1(mu[2], alpha)
    pi02 = price_profits1(phi02,mu[2],alpha)[2][1]
    
    obj1(phi0) = -1*nash_in_nash( [phi0[1], phi[2]] , mu, alpha;pi0=pi02)[1]
    res1 =  optimize(obj1, [phi[1]], BFGS())

    obj2(phi0) = -1*nash_in_nash( [phi[1], phi0[1]] , mu, alpha;pi0=pi01)[2]
    res2 =  optimize(obj2, [phi[2]], BFGS())
    
    return [ res1.minimizer[1], res2.minimizer[1] ]
end

println( bargain_helper([13.,13.], [1.,1.] , .1) )

[9.86094, 9.86094]


In [73]:
function simult_bargain(mu, alpha)
    """solve the bargaining problems seperately and iterate for a solution"""       
    
    #loop variables
    diff =  5.
    phi = [3., 3.]

    maxiter = 10
    while (maxiter >=0) & (diff > 10e-7)
        #seems as though there is a contraction mapping here, need to think more about why
        phi_new = bargain_helper(phi, mu, alpha)
        
        #update loop variables
        diff = maximum( abs.(phi - phi_new) )
        maxiter = maxiter-1
        
        #update phi
        phi = phi_new
    end
    return phi
end
    
print(simult_bargain([.5,.5], 1.) )

[1.99667, 1.99667]

In [74]:
function make_table(steps, mu)
    """the argument is 
    steps = min_arg, max_arg, no_step """

    min_arg, max_arg, no_step = steps    
    step_size = (max_arg - min_arg)/no_step
    indep_var = min_arg:step_size:max_arg
    result1 = []

    for alpha=indep_var

        phi = [0. 0.]
        phi = simult_bargain(mu, alpha)
        price, profits = price_profits(phi,mu,alpha)
        share = market_shares(price, alpha)
        hosp = sum(mu .* share .* phi)

        append!(result1, [[alpha, phi[1], share[1], price[1], profits[1], hosp ]] ) 
    end
    
    println("\n")
    for line =result1
        print( round(line[1],sigdigits=3) )
        for i=2:6
            print( " & ")
            print(round(line[i],sigdigits=3) )
        end
        print(" \\\\\\\\ \n")
    end   
    print('\n')
        
   
    

end

    
make_table( (.5,1,5), [.5,.5] )



0.5 & 3.99 & 0.0978 & 4.21 & 0.217 & 0.391 \\\\ 
0.6 & 3.33 & 0.0978 & 3.51 & 0.181 & 0.326 \\\\ 
0.7 & 2.85 & 0.0978 & 3.01 & 0.155 & 0.279 \\\\ 
0.8 & 2.5 & 0.0978 & 2.63 & 0.136 & 0.244 \\\\ 
0.9 & 2.22 & 0.0978 & 2.34 & 0.12 & 0.217 \\\\ 
1.0 & 2.0 & 0.0978 & 2.11 & 0.108 & 0.195 \\\\ 

